# **Purpose: Collect News Article titles about a currency pair and get its associated price movement (Up or Down) **

****

In [6]:
!pip install beautifulsoup4 requests

**Given a currency pair, get relevant news articles from the previous day**

In [7]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from urllib.request import Request, urlopen
from tqdm import tqdm

class fx_news_scraper():
    def __init__(self):
        self.number_of_pages = 29
        self.news_list = []
        self.target_url = 'https://www.investing.com/news/forex-news/'
        self.output_csv = 'app/data/fx-news-archive_test.csv'
        self.news_dataset = pd.DataFrame()

    def scraper(self):
        for page_no in tqdm(range(1, self.number_of_pages+1)):
            url = self.target_url + str(page_no)
            req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
            
            webpage = urlopen(req).read()
            
            soup = bs(webpage, "html.parser")
            
            news_container_div = soup.find('div', class_= 'news-analysis-v2_articles-container__3fFL8 mdMax:px-3 mb-12')

            try:
                for news_div in news_container_div.find_all('div', class_= 'news-analysis-v2_content__z0iLP w-full text-xs sm:flex-1'):
                    news_title_a = news_div.find('a', class_ = 'text-inv-blue-500 hover:text-inv-blue-500 hover:underline focus:text-inv-blue-500 focus:underline whitespace-normal text-sm font-bold leading-5 !text-[#181C21] sm:text-base sm:leading-6 lg:text-lg lg:leading-7')
                    news_body    = news_div.find('p')
                    news_date    = news_div.find('time', class_ = 'mx-1 shrink-0 text-xs leading-4').get('datetime')
                    
                    news_href    = news_title_a.get('href')
                    
                    self.news_list.append([page_no, news_date, news_title_a.text , news_body.text, news_href])

                    
            except ValueError as vEr:
                print(vEr)

            
            self.news_dataset = pd.DataFrame(self.news_list, columns = ['page', 'date','title','body', 'url'] )
            self.news_dataset.to_csv(self.output_csv)
            
            page_no += 1

        print ('scraping completed')
        print (f'output --> {self.output_csv}')

In [8]:
news_scraper = fx_news_scraper()
news_scraper.scraper()

  0%|          | 0/29 [00:00<?, ?it/s]


HTTPError: HTTP Error 403: Forbidden

In [61]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def get_forex_news_five_years(currency_pair):

    headers = {"User-Agent": "Mozilla/5.0"}
    articles = []
    
    page_count = 0
    stop_flag = False
    article_titles = []
    article_dates = []
    while not stop_flag and page_count < 5:
        if page_count == 0:
            url = 'https://www.investing.com/currencies/'+currency_pair+'-news'
        else:
            url = 'https://www.investing.com/currencies/'+currency_pair+'-news/'+str(page_count)
        response = requests.get(url, headers=headers)
        print('url:',url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            news_list = soup.find('ul', attrs={'data-test': 'news-list'})

            # Extract all <article> elements inside it
            articles = news_list.find_all('article')
            for article in articles:
                div_tag = article.find('div', class_='block w-full sm:flex-1')
                if div_tag:
                    # Extract the <a> tag inside the <div>
                    a_tag = div_tag.find('a', href=True)
                else:
                    print("coudln't find div tag of class block w-full sm:flex-1")
                    assert(False)
                if a_tag:
                    article_title = a_tag.get_text(strip=True)
                    article_titles.append(article_title)
                else:
                    print("coudln't find a tag with href")
                    assert(False)

                time_tag = div_tag.find('time', datetime=True)

                if time_tag:
                    # Get the datetime attribute from the <time> tag
                    datetime_value = time_tag['datetime'].split()[0]
                    article_dates.append(datetime_value)
                    dt = datetime.strptime(datetime_value, '%Y-%m-%d').date()
                    current_date = datetime.now().date()
                    diff_in_years = current_date.year - dt.year
                    print("years diff:",diff_in_years)
                    if diff_in_years == 5:
                        stop_flag = True
                else:
                    print("No <time> tag with datetime attribute found.")
                    assert(False)

        else:
            print("couldn't get page info:",url)
            assert(False)

        page_count += 1

    df = pd.DataFrame({
    'Title': article_titles,
    'Date': article_dates
    })
    c1 = currency_pair.split('-')[0]
    c2 = currency_pair.split('-')[1]
    str_ = c1+'_'+c2
    df.to_csv('currency_pair_info'+str_+'.csv', index=False)

currency_pair = 'gbp-usd'
# Example Usage
news_articles = get_forex_news_five_years(currency_pair)
print('done extracting news articles')
print(news_articles)
for article in news_articles:
    print(f"Title: {article['title']}\nLink: {article['link']}\nContent: {article['content'][:500]}...\n")


for page 0
url: https://www.investing.com/currencies/gbp-usd-news
parsed page successfuly
title: US dollar rallies; Aussie hits five-year low after China tariff concerns
Datetime: 2025-04-04

years diff: 0
title: Dollar bounces off lows; euro slips after German data
Datetime: 2025-04-04

years diff: 0
title: Dollar sinks as investors grapple with tariff aftermath
Datetime: 2025-04-03

years diff: 0
title: U.K. stocks lower at close of trade; Investing.com United Kingdom 100 down 1.56%
Datetime: 2025-04-03

years diff: 0
title: Dollar retreats as tariffs lift recession fears; euro gains
Datetime: 2025-04-03

years diff: 0
title: US dollar falls vs yen, euro holds gains after Trump announces new tariffs
Datetime: 2025-04-02

years diff: 0
title: U.K. stocks lower at close of trade; Investing.com United Kingdom 100 down 0.34%
Datetime: 2025-04-02

years diff: 0
title: Dollar steady ahead of key tariffs announcement; euro slips
Datetime: 2025-04-02

years diff: 0
title: Forex de-risking ex

KeyboardInterrupt: 

In [20]:
!pip install selenium webdriver-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 75.9 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 50.4 MB/s eta 0:00:00


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Configure Chrome to run headless (optional)
options = Options()
options.add_argument("--headless")  # Run without opening a browser
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open a test page
driver.get("https://www.google.com")

# Print the page title
print("Page Title:", driver.title)

# Close the driver
driver.quit()

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55c992b3a4e3 <unknown>
#1 0x55c992869c76 <unknown>
#2 0x55c992890757 <unknown>
#3 0x55c99288f029 <unknown>
#4 0x55c9928cdccc <unknown>
#5 0x55c9928cd47f <unknown>
#6 0x55c9928c4de3 <unknown>
#7 0x55c99289a2dd <unknown>
#8 0x55c99289b34e <unknown>
#9 0x55c992afa3e4 <unknown>
#10 0x55c992afe3d7 <unknown>
#11 0x55c992b08b20 <unknown>
#12 0x55c992aff023 <unknown>
#13 0x55c992acd1aa <unknown>
#14 0x55c992b236b8 <unknown>
#15 0x55c992b23847 <unknown>
#16 0x55c992b33243 <unknown>
#17 0x7fa8551ca609 start_thread


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Setup Selenium with ChromeDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (no GUI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL of FXStreet News
url = "https://www.fxstreet.com/"  # Change to actual news page if needed

# Open the page
driver.get(url)

# Wait until the JavaScript-rendered content loads
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "ais-hits"))
)

# Get fully rendered HTML
html = driver.page_source

# Parse with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Find the news articles container
news_container = soup.find("div", class_="ais-hits")

if news_container:
    print("Found 'ais-hits' container!")
    print(news_container.prettify())  # Print extracted content
else:
    print("Could not find 'ais-hits' container.")

# Save the parsed page (for debugging)
with open("fxstreet_news.html", "w", encoding="utf-8") as f:
    f.write(soup.prettify())

# Close Selenium
driver.quit()


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55d3580a34e3 <unknown>
#1 0x55d357dd2c76 <unknown>
#2 0x55d357df9757 <unknown>
#3 0x55d357df8029 <unknown>
#4 0x55d357e36ccc <unknown>
#5 0x55d357e3647f <unknown>
#6 0x55d357e2dde3 <unknown>
#7 0x55d357e032dd <unknown>
#8 0x55d357e0434e <unknown>
#9 0x55d3580633e4 <unknown>
#10 0x55d3580673d7 <unknown>
#11 0x55d358071b20 <unknown>
#12 0x55d358068023 <unknown>
#13 0x55d3580361aa <unknown>
#14 0x55d35808c6b8 <unknown>
#15 0x55d35808c847 <unknown>
#16 0x55d35809c243 <unknown>
#17 0x7f08b32a1609 start_thread
